## Static Train/Val/Test division

In [ ]:
# Check the Division Configuration Parameters
import numpy as np
import Functions.MyDataset as MyDataset
import Spartan.support_modules as MySupport


In [ ]:
MyDataset.get_pat_splits(static=True)

In [ ]:
# file name format: {name}_{size}_VolPts_{id}.mat (AH_17617648_VolPts.mat)
pat_names = ['AH', 'AZ', 'DE', 'DM', 'DM2', 'DGL', 'FA', 'GE', 'GM', 'GP', 'HB', 'HH',
             'JH', 'JM', 'LG', 'LP', 'MJ', 'NV', 'PH', 'SM']

# pat_splits = MyDataset.get_pat_splits(static=True)
pat_splits = []

if not pat_splits:
    np.random.shuffle(pat_names)
    name_splits = np.split(pat_names, [int(.7 * len(pat_names)), int(.8 * len(pat_names))])
else:
    name_splits = [[pat_names[i] for i in pat_splits[0]],
                   [pat_names[i] for i in pat_splits[1]], [pat_names[i] for i in pat_splits[2]]]

print(name_splits)

In [ ]:
import importlib

importlib.reload(MyDataset)

#pat_splits = MyDataset.get_pat_splits(static=True)
pat_splits = []

if not pat_splits:
    # attention: this division is based on augmentation, no on patients (not sure if it will cause issues)
    idx_list = np.arange(0, 2000, 2)
    np.random.shuffle(idx_list)
    idx_splits = np.split(idx_list, [int(.7 * len(idx_list)), int(.8 * len(idx_list))])
else:
    idx_splits = [[list(range(i*100, i*100+100, 2)) for i in j] for j in pat_splits]
    for i in range(0, 3):
        idx_splits[i] = [num for sublist in idx_splits[i] for num in sublist]
        np.random.shuffle(idx_splits[i])

print(idx_splits)

## Check Test Dataset prediction results

In [ ]:
import numpy as np

In [ ]:
res_dir = "/Users/achs/PhD/code/CT-MRI_LandmarkDetection/Spartan/Output/y_test_pred/"

test_y_pred_file = "final_first_model_y_test.npy"
# test_y_pred_file = "bestVal_first_model_y_test.npy"

test_y_pred_path = res_dir + test_y_pred_file
test_y_pred = np.load(test_y_pred_path)


In [ ]:
import Spartan.support_modules as supporter
import Functions.MyDataset as MyDataset

# Get the Test Dataset Prediction Results
size = (176, 176, 48)
with_res = True

str_size = str(size[0]) + "_" + str(size[1]) + "_" + str(size[2])
if with_res:
    str_size = str_size + "_PD"

X_test, Y_test, res_test = \
    supporter.load_dataset("/Volumes/Shawn_HDD/PhD/Project/Date/augmentation_from_matlab/Rescaled/" + str_size + "/",
                           size, pat_splits=MyDataset.get_pat_splits(static=True), with_res=with_res,
                           only_test=True)

In [ ]:
import tensorflow as tf

rep_res = tf.repeat(res_test, 4, axis=1)

In [ ]:
print(rep_res.shape)
print(rep_res[1])

In [ ]:
Y_test_mapped = Y_test * rep_res / 0.15

In [ ]:
test_y_pred_mapped = test_y_pred * rep_res / 0.15

In [ ]:
print(test_y_pred_mapped.shape)
print(test_y_pred_mapped[0])

## check the difference between Predictions and Ground Truth

In [ ]:
# 1. Total error distance
# 2. Maximum one point's error distance
# 3. If it can involve the Ground Truth points
# 4. If it can involve the Target Area

# 1. Total error distance
err_diff = Y_test_mapped - test_y_pred_mapped
square_err_diff = tf.pow(err_diff, 2)
sum_square_err_diff = tf.reduce_sum(square_err_diff, axis=[1, 2])

In [ ]:
print(sum_square_err_diff.shape)
max_err_idx = np.argmax(sum_square_err_diff, axis=0)
print("max error idx: ", max_err_idx, "value: ", sum_square_err_diff[max_err_idx])

In [ ]:
idx = 187
print(Y_test_mapped[idx])
print(test_y_pred_mapped[idx])

In [ ]:
pat_names = MyDataset.get_pat_names()
test_pat_idx = MyDataset.get_pat_splits(static=True)[2] # 0: train, 1: val, 2: test

In [ ]:
test_pt_idx = np.floor(idx / 50).astype(int)
pt_aug_id = idx % 50 + 1

In [ ]:
pat_name = pat_names[test_pat_idx[test_pt_idx]]

In [ ]:
## Check the cropped results
import numpy as np
import importlib
import Functions.MyDataset as MyDataset
import Functions.Visualization as Visualization

JH_aug_mat_path = "/Volumes/Shawn_HDD/PhD/Project/Date/augmentation_from_matlab/original_augmentation_data/LP_aug_38.mat"
# JH_reshape_vol_mat_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Val/Input/FA_17017030_AugVol_2.mat"
# JH_reshape_pts_mat_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Val/Output/FA_17017030_AugPts_2.mat"

pixel_space = [0.15, 0.15, 0.15]

importlib.reload(MyDataset)

JH_aug_volume, JH_aug_pts, _ = MyDataset.load_mat_data(JH_aug_mat_path)

Visualization.show_pts(JH_aug_volume, test_y_pred_mapped[idx], pixel_space)

In [ ]:
print(X_test[0,:,:,:,0].shape)

In [ ]:
Visualization.show_pts(X_test[idx,:,:,:,0], test_y_pred[idx], pixel_space)